#Pre-processing

##Import neccesary libraries

In [1]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.8 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from datasets import load_dataset
from transformers import BartTokenizer
import numpy as np
import re
import random
from nltk.corpus import stopwords
import nltk
from bs4 import BeautifulSoup
import pandas as pd

# Download NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Set random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


##Load the data

In [3]:
# load the cnn-dm dataset using dataset library
cnn_dm_dataset = load_dataset('cnn_dailymail', '3.0.0')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [4]:
cnn_dm_dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [5]:
pd.DataFrame(cnn_dm_dataset['train'][:100]).head()

,article,highlights,id
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,ee8871b15c50d0db17b0179a6d2beab35065f1e9
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",06352019a19ae31e527f37f7571c6dd7f0c5da37
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a


In [63]:
TRAIN_SIZE = 100000
VAL_SIZE = 10000
TEST_SIZE = 10000

In [6]:
# train_df = pd.DataFrame(cnn_dm_dataset['train'][:TRAIN_SIZE])
# val_df = pd.DataFrame(cnn_dm_dataset['validation'][:VAL_SIZE])
# test_df = pd.DataFrame(cnn_dm_dataset['test'][:TEST_SIZE])
train_df = pd.DataFrame(cnn_dm_dataset['train'])
val_df = pd.DataFrame(cnn_dm_dataset['validation'])
test_df = pd.DataFrame(cnn_dm_dataset['test'])

##Pre-processing dataset

In [14]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [15]:
def clean_entries(df):
    """Clean null entries in the DataFrame."""
    df.drop_duplicates(subset=['article'], inplace=True) # Drop duplicates
    df.dropna(axis=0,inplace=True) # Drop rows with missing values

    return df

In [16]:
def clean_text(text):
    """Clean the text by removing extra whitespace, special characters, and stopwords."""
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with single space
    text = re.sub(r'[^\w\s.,!?]', '', text)  # Remove special characters except punctuation
    text = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in text.split(" ")])
    words = text.split()
    words = [word for word in words if word.lower() not in stop_words]  # Remove stopwords
    return ' '.join(words).strip()

In [17]:
def preprocess_data(df):
    """Preprocess the dataframe"""
    # Drop null values
    df = clean_entries(df)

    # Clean text and summaries
    df['article'] = df['article'].apply(clean_text)
    df['highlights'] = df['highlights'].apply(clean_text)

    # Add start and end tokens to summaries
    df['highlights'] = '<START>' + df['highlights'] + '<END>'

    return df

In [12]:
train_df = preprocess_data(train_df)
val_df = preprocess_data(val_df)
test_df = preprocess_data(test_df)

In [13]:
train_df['highlights'][98]

'<START>YouTube questions address taxes, Bible, abortion, gun control . Giuliani, Romney, Huckabee spar immigration . McCain challenges Paul suggestion bring troops home Iraq . Nearly 5,000 videos GOP debate 2,000 Democratic debate .<END>'

In [18]:
# Initialize the tokenizer (BART tokenizer in this case, adjust based on your model)
#tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Set maximum lengths for input and output
MAX_LEN_TEXT = 200
MAX_LEN_SUMMARY = 50

In [15]:

def create_tokenizer(texts, vocab_size):
    """Create and fit tokenizer"""
    tokenizer = Tokenizer(oov_token='<OOV>')
    tokenizer.fit_on_texts(texts)
    return tokenizer

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer
SPECIAL_TOKENS = {
    'PAD': '<PAD>',  # Padding token
    'START': '<START>',  # Start of sequence token
    'END': '<END>',  # End of sequence token
    'UNK': '<UNK>'   # Unknown token
}

class CustomTokenizer:
    def __init__(self, vocab_size):
        self.vocab_size = vocab_size
        self.tokenizer = Tokenizer(num_words=vocab_size, oov_token=SPECIAL_TOKENS['UNK'])

    def fit_on_texts(self, texts):
        # Add special tokens to the texts to ensure they're in vocabulary
        special_text = [' '.join(SPECIAL_TOKENS.values())]
        all_texts = special_text + list(texts)

        # Fit tokenizer on all texts
        self.tokenizer.fit_on_texts(all_texts)

        # Make sure special tokens are in vocabulary by updating word_index
        for token in SPECIAL_TOKENS.values():
            if token not in self.tokenizer.word_index:
                # Add to word_index with a new index
                next_index = max(self.tokenizer.word_index.values()) + 1
                self.tokenizer.word_index[token] = next_index

        # Update index_word dictionary
        self.tokenizer.index_word = {v: k for k, v in self.tokenizer.word_index.items()}

    def texts_to_sequences(self, texts):
        return self.tokenizer.texts_to_sequences(texts)

    def sequences_to_texts(self, sequences):
        return self.tokenizer.sequences_to_texts(sequences)

    @property
    def word_index(self):
        return self.tokenizer.word_index

    @property
    def index_word(self):
        return self.tokenizer.index_word

def create_tokenizer_with_special_tokens(texts, vocab_size):
    """Create and fit tokenizer with special tokens"""
    tokenizer = CustomTokenizer(vocab_size)
    tokenizer.fit_on_texts(texts)
    return tokenizer

In [17]:
class SequenceProcessor:
    def __init__(self, tokenizer, max_len_text, max_len_summary):
        self.tokenizer = tokenizer
        self.max_len_text = max_len_text
        self.max_len_summary = max_len_summary
        # Create reverse word index as integers
        self.reverse_word_index = {int(k): v for k, v in tokenizer.index_word.items()}
        self.start_token_id = tokenizer.word_index[SPECIAL_TOKENS['START']]
        self.end_token_id = tokenizer.word_index[SPECIAL_TOKENS['END']]
        self.pad_token_id = tokenizer.word_index[SPECIAL_TOKENS['PAD']]

    def prepare_sequences(self, texts, summaries=None, training=True):
        """
        Prepare sequences for model input with proper padding
        """
        # Convert texts to sequences and pad
        text_sequences = self.tokenizer.texts_to_sequences(texts)
        input_ids = pad_sequences(
            text_sequences,
            maxlen=self.max_len_text,
            padding='post',
            truncating='post'
        )

        if not training or summaries is None:
            return input_ids, None

        # Convert summaries to sequences and pad
        summary_sequences = self.tokenizer.texts_to_sequences(summaries)
        target_ids = pad_sequences(
            summary_sequences,
            maxlen=self.max_len_summary,
            padding='post',
            truncating='post'
        )

        # Create decoder input by shifting target ids right
        decoder_input_ids = np.zeros_like(target_ids)
        decoder_input_ids[:, 1:] = target_ids[:, :-1]
        decoder_input_ids[:, 0] = self.start_token_id

        # Create decoder target
        decoder_target_ids = np.zeros((target_ids.shape[0], target_ids.shape[1], 1))
        decoder_target_ids[:, :, 0] = target_ids

        return input_ids, decoder_input_ids, decoder_target_ids

    def decode_sequences(self, sequences):
        """
        Convert token sequences back to text
        """
        texts = []
        for seq in sequences:
            # Convert numpy array to list of integers and remove padding
            token_ids = [int(i) for i in seq if i != 0]
            # Convert indices to tokens
            tokens = [self.reverse_word_index.get(idx, '') for idx in token_ids]
            # Join tokens and clean up special tokens
            text = ' '.join(tokens)
            text = text.replace('<START>', '').replace('<END>', '').strip()
            texts.append(text)
        return texts

In [18]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
def prepare_sequences(texts, summaries, tokenizer, max_len_text, max_len_summary):
    """Convert texts and summaries to padded sequences"""
    # Convert to sequences
    text_sequences = tokenizer.texts_to_sequences(texts)
    summary_sequences = tokenizer.texts_to_sequences(summaries)

    # Pad sequences
    text_padded = pad_sequences(text_sequences, maxlen=max_len_text, padding='post')
    summary_padded = pad_sequences(summary_sequences, maxlen=max_len_summary, padding='post')

    return text_padded, summary_padded

In [19]:
# Create tokenizer
VOCAB_SIZE = 30000
all_texts = list(train_df['article']) + list(train_df['highlights'])
tokenizer = create_tokenizer_with_special_tokens(all_texts, VOCAB_SIZE)

In [20]:
tokenizer.word_index.get('<START>',1)

369387

In [21]:
# Initialize the sequence processor
seq_processor = SequenceProcessor(tokenizer, MAX_LEN_TEXT, MAX_LEN_SUMMARY)

# Prepare sequences for training
input_ids, decoder_input_ids, decoder_target_ids = seq_processor.prepare_sequences(
    train_df['article'],
    train_df['highlights']
)

# Prepare validation sequences
val_input_ids, val_decoder_input_ids, val_decoder_target_ids = seq_processor.prepare_sequences(
    val_df['article'],
    val_df['highlights']
)

test_input_ids, test_decoder_input_ids, test_decoder_target_ids = seq_processor.prepare_sequences(
    test_df['article'],
    test_df['highlights']
)

In [22]:
print(input_ids.shape)
print(decoder_input_ids.shape)
print(decoder_target_ids.shape)
print(val_input_ids.shape)
print(val_decoder_input_ids.shape)
print(val_decoder_target_ids.shape)

(96914, 200)
(96914, 50)
(96914, 50, 1)
(5000, 200)
(5000, 50)
(5000, 50, 1)


In [23]:
input_ids[0]

array([  284,   718,  6962,  2160,  5868,   542,  2328, 16673,  5153,
         665,   121,   198,    68,  3057,    75,    68,  4863,  2575,
         701,   120,  4463,   178,   721,  1952,  6436,   242,  2328,
       16673,  2160,  5868,  2160,  5868,   477,  4168,  5894,  9075,
       26679,    69,    24,   171,  1413,    10,   567,     1,  1790,
         152,  1383,  1107,  2623,  2600,  1153,    47,   406,     3,
           4,  3996,   613,   713,   701,  2946,  1141,  1356,   927,
         251,  2224,   154,   700,    17,  1324, 17291,   244,   158,
          47,    51,  1184,  1125, 14132,   181,    21,  2995,   181,
         771,   107,  1751,  1405, 14995, 13620,   701, 16673,   241,
       10813,  6516,  1141,  2623,  7443,    55,  3896,   454, 18694,
          81,  1949,  1000,   231,  1166,   157,     3,   716,   864,
        1736,   145,  7841,   882,  1960,   675,  5150,  2494, 13000,
        1835,  6894,   835,   567,  1184,  2308,  1597,   176,     2,
         563,  3775,

In [39]:
# prompt: write code to decode a sequence

decoded_text = seq_processor.decode_sequences([input_ids[0]])
print(decoded_text)
print(train_df['article'][0])

['london england reuters harry potter star daniel radcliffe gains access reported 20 million 41 1 million fortune turns 18 monday insists money wont cast spell him daniel radcliffe harry potter harry potter order phoenix disappointment gossip columnists around world young actor says plans <UNK> cash away fast cars drink celebrity parties dont plan one people who soon turn 18 suddenly buy massive sports car collection something similar told australian interviewer earlier month dont think ill particularly extravagant things like buying things cost 10 pounds books cds dvds 18 radcliffe able gamble casino buy drink pub see horror film hostel part ii currently six places number one movie uk box office chart details hell mark landmark birthday wraps agent publicist comment plans ill definitely sort party said interview hopefully none reading it <UNK> earnings first five potter films held trust fund able touch despite growing fame riches actor says keeping feet firmly ground people always loo

#Initial se2seq training

In [42]:
!pip install --q rouge-score evaluate bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.3 MB/s eta 0:00:00


In [70]:
from tensorflow.keras.callbacks import Callback
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
import numpy as np
import tensorflow as tf

class SummarizationMetricsCallback(Callback):
    def __init__(self, seq_processor, validation_data, eval_frequency=5):
        super().__init__()
        self.seq_processor = seq_processor
        self.x_val, self.y_val = validation_data
        self.eval_frequency = eval_frequency
        self.scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        self.smoothing = SmoothingFunction().method1
        self.history = {
            'val_rouge1': [],
            'val_rouge2': [],
            'val_rougeL': [],
            'val_bleu': []
        }

    def decode_sequence(self, input_seq):
        """Decode a single input sequence using the current model"""
        input_seq = np.expand_dims(input_seq, axis=0)
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = self.seq_processor.start_token_id

        decoded_indices = []

        for i in range(self.seq_processor.max_len_summary):
            output_tokens = self.model.predict([input_seq, target_seq], verbose=0)
            sampled_token_index = np.argmax(output_tokens[0, -1, :])

            if sampled_token_index == 0 or sampled_token_index == self.seq_processor.end_token_id:
                break

            decoded_indices.append(int(sampled_token_index))

            target_seq = np.zeros((1, 1))
            target_seq[0, 0] = sampled_token_index

        return self.seq_processor.decode_sequences([decoded_indices])[0]

    def calculate_metrics(self, reference, hypothesis):
        """Calculate both ROUGE and BLEU metrics"""
        # Calculate ROUGE scores
        rouge_scores = self.scorer.score(reference, hypothesis)

        # Calculate BLEU score
        reference_tokens = reference.split()
        hypothesis_tokens = hypothesis.split()
        bleu_score = sentence_bleu([reference_tokens], hypothesis_tokens,
                                 smoothing_function=self.smoothing)

        return {
            'rouge1': rouge_scores['rouge1'].fmeasure,
            'rouge2': rouge_scores['rouge2'].fmeasure,
            'rougeL': rouge_scores['rougeL'].fmeasure,
            'bleu': bleu_score
        }

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.eval_frequency == 0:
            print(f"\nCalculating ROUGE and BLEU metrics for epoch {epoch+1}...")

            # Evaluate on a subset of validation data for speed
            num_eval_samples = min(100, len(self.x_val))

            epoch_scores = {
                'rouge1': [], 'rouge2': [], 'rougeL': [], 'bleu': []
            }

            for i in range(num_eval_samples):
                # Generate prediction
                pred_text = self.decode_sequence(self.x_val[i])

                # Get reference summary
                ref_text = self.seq_processor.decode_sequences([self.y_val[i]])[0]

                # Calculate metrics
                metrics = self.calculate_metrics(ref_text, pred_text)

                # Accumulate scores
                for key in epoch_scores:
                    epoch_scores[key].append(metrics[key])

            # Calculate average scores
            avg_scores = {
                'val_rouge1': np.mean(epoch_scores['rouge1']),
                'val_rouge2': np.mean(epoch_scores['rouge2']),
                'val_rougeL': np.mean(epoch_scores['rougeL']),
                'val_bleu': np.mean(epoch_scores['bleu'])
            }

            # Update history
            for key in self.history:
                self.history[key].append(avg_scores[key])

            # Print current scores
            print("\nValidation Metrics:")
            print(f"ROUGE-1: {avg_scores['val_rouge1']:.4f}")
            print(f"ROUGE-2: {avg_scores['val_rouge2']:.4f}")
            print(f"ROUGE-L: {avg_scores['val_rougeL']:.4f}")
            print(f"BLEU: {avg_scores['val_bleu']:.4f}")

            # Print an example
            print("\nExample Generation:")
            print(f"Reference: {ref_text}")
            print(f"Generated: {pred_text}\n")

In [30]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

In [71]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Concatenate, Input
from tensorflow.keras.models import Model

logger = tf.get_logger()

class AttentionLayer(tf.keras.layers.Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
    """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)

        # Create a trainable weight variable for this layer.
        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)

    def call(self, inputs):
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs

        def energy_step(inputs, states):
            encoder_full_seq = states[-1]
            W_a_dot_s = K.dot(encoder_full_seq, self.W_a)
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            e_i = K.softmax(e_i)
            return e_i, [e_i]

        def context_step(inputs, states):
            encoder_full_seq = states[-1]
            c_i = K.sum(encoder_full_seq * K.expand_dims(inputs, -1), axis=1)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)

        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e], constants=[encoder_out_seq]
        )
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c], constants=[encoder_out_seq]
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

K.clear_session()
latent_dim = 500

# Encoder
encoder_inputs = Input(shape=(MAX_LEN_TEXT,))
enc_emb = Embedding(VOCAB_SIZE, latent_dim,trainable=True)(encoder_inputs)

#LSTM 1
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

#LSTM 2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

#LSTM 3
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# Set up the decoder.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(VOCAB_SIZE, latent_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

#LSTM using encoder_states as initial state
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

#Attention Layer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# Concat attention output and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#Dense layer
decoder_dense = TimeDistributed(Dense(VOCAB_SIZE, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 200)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 200, 500)       │     15,000,000 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ [(None, 200, 500),     │      2,002,000 │ embedding[0][0]        │
│                           │ (None, 500), (None,    │                │                        │
│                           │ 500)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, 200, 500),     │      2,002,000 │ lstm[0][0]             │
│                           │ (None, 500), (None,    │                │                        │
│                           │ 500)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, None, 500)      │     15,000,000 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_2 (LSTM)             │ [(None, 200, 500),     │      2,002,000 │ lstm_1[0][0]           │
│                           │ (None, 500), (None,    │                │                        │
│                           │ 500)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_3 (LSTM)             │ [(None, None, 500),    │      2,002,000 │ embedding_1[0][0],     │
│                           │ (None, 500), (None,    │                │ lstm_2[0][1],          │
│                           │ 500)]                  │                │ lstm_2[0][2]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_layer           │ [(None, None, 500),    │        500,500 │ lstm_2[0][0],          │
│ (AttentionLayer)          │ (None, None, 200)]     │                │ lstm_3[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concat_layer              │ (None, None, 1000)     │              0 │ lstm_3[0][0],          │
│ (Concatenate)             │                        │                │ attention_layer[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed          │ (None, None, 30000)    │     30,030,000 │ concat_layer[0][0]     │
│ (TimeDistributed)         │                        │                │                        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 68,538,500 (261.45 MB)

 Trainable params: 68,538,500 (261.45 MB)

 Non-trainable params: 0 (0.00 B)

In [72]:
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Define learning rate schedule
initial_learning_rate = 0.001
decay_steps = 10000
decay_rate = 0.9

lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps,
    decay_rate,
    staircase=True
)

# Compile the model with the learning rate schedule
optimizer = Adam(learning_rate=lr_schedule)
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [73]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)
# Initialize callbacks
metrics_callback = SummarizationMetricsCallback(
    seq_processor=seq_processor,
    validation_data=(val_input_ids, val_decoder_target_ids),
    eval_frequency=5
)

In [74]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the model checkpoint callback
checkpoint_callback = ModelCheckpoint(
    filepath='model_checkpoints/seq2seq_model_epoch_{epoch:02d}.keras',  # File path for saving model
    save_weights_only=False,  # Whether to save only weights or the whole model
    monitor='val_loss',  # Monitor validation loss to save best models (optional)
    save_best_only=True,  # If True, only the model with the best performance will be saved
    save_freq='epoch',  # Save model at the end of each epoch. You can also set a specific number of batches
    verbose=1  # Print messages when saving the model
)

In [ ]:
# Update model.fit call
history = model.fit(
    [input_ids, decoder_input_ids],
    decoder_target_ids,
    epochs=50,
    batch_size=512,
    callbacks=[es, checkpoint_callback, metrics_callback],
    validation_data=([val_input_ids, val_decoder_input_ids], val_decoder_target_ids),
    verbose=1
)

In [35]:
metrics_callback.history

{'val_rouge1': [0.04574278403394377],
 'val_rouge2': [0.0017159367012716542],
 'val_rougeL': [0.04574278403394377],
 'val_bleu': [0.004273190420209166]}

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()


In [43]:
test_input_ids, test_decoder_input_ids, test_decoder_target_ids = seq_processor.prepare_sequences(
    test_df['article'],
    test_df['highlights']
)

In [ ]:
# prompt: write code to test the model

# Function to generate summaries for given input texts
def generate_summaries(input_texts, model, seq_processor):
    """Generates summaries for a list of input texts using the trained model"""
    input_ids, _ = seq_processor.prepare_sequences(input_texts, training=False)

    summaries = []
    for i in range(len(input_ids)):
        decoded_summary = decode_sequence(input_ids[i], model, seq_processor)
        summaries.append(decoded_summary)

    return summaries


# Function to decode a sequence using the model
def decode_sequence(input_seq, model, seq_processor):
    """Decodes a single input sequence using the model"""
    input_seq = np.expand_dims(input_seq, axis=0)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = seq_processor.start_token_id

    decoded_indices = []

    for i in range(seq_processor.max_len_summary):
        output_tokens = model.predict([input_seq, target_seq], verbose=0)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])

        if sampled_token_index == 0 or sampled_token_index == seq_processor.end_token_id:
            break

        decoded_indices.append(int(sampled_token_index))

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

    return seq_processor.decode_sequences([decoded_indices])[0]


# Test the model with a few examples from the validation set
num_test_samples = 5  # Number of samples to test
for i in range(num_test_samples):
    input_text = test_df['article'].iloc[i]
    reference_summary = test_df['highlights'].iloc[i]

    generated_summary = decode_sequence(test_input_ids[i], model, seq_processor)

    print(f"Input Text: {input_text}")
    print(f"Reference Summary: {reference_summary}")
    print(f"Generated Summary: {generated_summary}")
    print("-" * 50)
